<a href="https://colab.research.google.com/github/habroland/Blog/blob/master/EquivalentLens/EquivalentLensCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Equivalent-lens system for an array of lenses. Check this <a href="https://underthemath.wordpress.com/2020/05/31/equivalent-lens-system/">blog</a> for details.

# Functions

In [1]:
def equivSingleLens_2lenses(f1,d12,f2):
    """ Calculates the equivalent single-lens system for an array of two lenses

    Arguments: (f1)---d12--->(f2)
    f1: Focal length of the first lens
    d12: Distance between the lenses
    f2: Focal length of second lens

    Returns: ---df--->(feff)---db--->
    df: Front distance
    feff: Effective focal length
    db: Back distance

    """
    df = (d12*f1)/(f1+f2-d12)
    feff = (f2*f1)/(f1+f2-d12)
    db = (d12*f2)/(f1+f2-d12)

    return df, feff, db

def equivSingleLens(fls,ds):
    """ Calculates the equivalent single-lens system for an array of lenses

    Arguments:
    fls: List of N focal lengths
    ds: List of N-1 distances, each one between lenses f_i and f_(i+1)

    Returns: Dictionary
    df: Front distance
    feff: Effective focal length
    db: Back distance

    """
    assert(len(fls) == len(ds)+1)

    df = 0
    feff = fls[0]
    db = 0
    for i in range(len(ds)):
        df_temp, feff, db = equivSingleLens_2lenses(feff,db+ds[i],fls[i+1])
        df += df_temp

    return {"df":df, "feff":feff, "db":db}

# Results

In [2]:
from sympy import *
init_printing()

## Variables

Four lenses, and the distances between consecutive pairs

(F1)---D12--->(F2)---D23--->(F3)---D34--->(F4)

In [3]:
F1, F2, F3, F4 = symbols('F1 F2 F3 F4')
D12, D23, D34 = symbols('D12 D23 D34')

## Subsection to test the functions

In [4]:
equivSingleLens_2lenses(F1,D12,F2)

⎛    D₁₂⋅F₁          F₁⋅F₂           D₁₂⋅F₂    ⎞
⎜──────────────, ──────────────, ──────────────⎟
⎝-D₁₂ + F₁ + F₂  -D₁₂ + F₁ + F₂  -D₁₂ + F₁ + F₂⎠

In [5]:
equivSingleLens([F1,F2],[D12])

{'df': D12*F1/(-D12 + F1 + F2),
 'feff': F1*F2/(-D12 + F1 + F2),
 'db': D12*F2/(-D12 + F1 + F2)}

In [6]:
doublet = equivSingleLens([F1,F2],[D12]) # https://realpython.com/iterate-through-dictionary-python/
for key in doublet:
  print(key, "->", doublet[key].subs({F1:-50, F2:100, D12:60}))

df -> 300
feff -> 500
db -> -600


In [7]:
# Compute 1/f2L
expand(simplify(1/(equivSingleLens([F1,F2],[D12])['feff'])))

   D₁₂    1    1 
- ───── + ── + ──
  F₁⋅F₂   F₂   F₁

## Compute df3, f3L and db3. Notice the pattern. Compute 1/f3L for my blog

In [16]:
simplify(equivSingleLens([F1,F2,F3],[D12,D23])['df'])

              F₁⋅(-D₁₂⋅D₂₃ + D₁₂⋅F₂ + D₁₂⋅F₃ + D₂₃⋅F₂)             
───────────────────────────────────────────────────────────────────
D₁₂⋅D₂₃ - D₁₂⋅F₂ - D₁₂⋅F₃ - D₂₃⋅F₁ - D₂₃⋅F₂ + F₁⋅F₂ + F₁⋅F₃ + F₂⋅F₃

In [13]:
expand(simplify(equivSingleLens([F1,F2,F3],[D12,D23])['feff']))

                             -F₁⋅F₂⋅F₃                              
────────────────────────────────────────────────────────────────────
-D₁₂⋅D₂₃ + D₁₂⋅F₂ + D₁₂⋅F₃ + D₂₃⋅F₁ + D₂₃⋅F₂ - F₁⋅F₂ - F₁⋅F₃ - F₂⋅F₃

In [19]:
simplify(expand(equivSingleLens([F1,F2,F3],[D12,D23])['db']))

              F₃⋅(-D₁₂⋅D₂₃ + D₁₂⋅F₂ + D₂₃⋅F₁ + D₂₃⋅F₂)             
───────────────────────────────────────────────────────────────────
D₁₂⋅D₂₃ - D₁₂⋅F₂ - D₁₂⋅F₃ - D₂₃⋅F₁ - D₂₃⋅F₂ + F₁⋅F₂ + F₁⋅F₃ + F₂⋅F₃

In [ ]:
expand(simplify(1/(equivSingleLens([F1,F2,F3],[D12,D23])['feff'])))

D₁₂⋅D₂₃     D₁₂     D₁₂     D₂₃     D₂₃    1    1    1 
──────── - ───── - ───── - ───── - ───── + ── + ── + ──
F₁⋅F₂⋅F₃   F₁⋅F₃   F₁⋅F₂   F₂⋅F₃   F₁⋅F₃   F₃   F₂   F₁

### Compute equivalent lens parameters in a mirrored system and compare with original
Both cases have the same effective focal length. The back distance of one case becomes the front distance of the mirrored instance, as expected.

(F1)---D12--->(F2)---D23--->(F3)

vs

(F3)---D23--->(F2)---D21--->(F1)

From a general math perspective, this symmetric behaviour is not granted, because matrix multiplication is not commutative, but it works for RTMs

In [ ]:
f3L = simplify(equivSingleLens([F1,F2,F3],[D12,D23])['feff'])
f3L_mir = simplify(equivSingleLens([F3,F2,F1],[D23,D12])['feff'])
simplify(f3L - f3L_mir)

0

In [ ]:
df3L = simplify(equivSingleLens([F1,F2,F3],[D12,D23])['df'])
db3L_mir = simplify(equivSingleLens([F3,F2,F1],[D23,D12])['db'])
simplify(df3L - db3L_mir)

0

In [ ]:
db3L = simplify(equivSingleLens([F1,F2,F3],[D12,D23])['db'])
df3L_mir = simplify(equivSingleLens([F3,F2,F1],[D23,D12])['df'])
simplify(db3L - df3L_mir)

0

## Compute 1/f4L

My code does the following, each step is bold:

for-loop step 1:

**(F1)---D12--->(F2)**---D23--->(F3)---D34--->(F4)

**---DF12--->(F12)---DB12--->**---D23--->(F3)---D34--->(F4)

for-loop step 2:

---DF12--->**(F12)---DB12--->---D23--->(F3)**---D34--->(F4)

---DF12--->**---DF123--->(F123)---DB123--->**---D34--->(F4)

for-loop step 3:

---DF12--->---DF123--->**(F123)---DB123--->---D34--->(F4)**

---DF12--->---DF123--->**---DF1234--->(F1234)---DB1234--->**

In [ ]:
f4L_inv=expand(simplify(1/(equivSingleLens([F1,F2,F3,F4],[D12,D23,D34])['feff'])))
f4L_inv

  D₁₂⋅D₂₃⋅D₃₄   D₁₂⋅D₂₃    D₁₂⋅D₂₃    D₁₂⋅D₃₄    D₁₂⋅D₃₄     D₁₂     D₁₂     D
- ─────────── + ──────── + ──────── + ──────── + ──────── - ───── - ───── - ──
  F₁⋅F₂⋅F₃⋅F₄   F₁⋅F₂⋅F₄   F₁⋅F₂⋅F₃   F₁⋅F₃⋅F₄   F₁⋅F₂⋅F₄   F₁⋅F₄   F₁⋅F₃   F₁

₁₂    D₂₃⋅D₃₄    D₂₃⋅D₃₄     D₂₃     D₂₃     D₂₃     D₂₃     D₃₄     D₃₄     D
─── + ──────── + ──────── - ───── - ───── - ───── - ───── - ───── - ───── - ──
⋅F₂   F₂⋅F₃⋅F₄   F₁⋅F₃⋅F₄   F₂⋅F₄   F₂⋅F₃   F₁⋅F₄   F₁⋅F₃   F₃⋅F₄   F₂⋅F₄   F₁

₃₄    1    1    1    1 
─── + ── + ── + ── + ──
⋅F₄   F₄   F₃   F₂   F₁

Do you see a pattern?
The last terms are the reciprocals of each focal length. The first term is the product of the distances divided by the product of the focal lengths, with a minus sign for an even number of lenses

Do you see a symmetry?
If you mirror the lens system, then you will get the same value. To mirror the system simply flip F1 with F4, F2 with F3, and D12 with D34.

### Compute 1/f4L in a different way

The first two and the last two lenses can be separately substituted by their equivalent lens system. The resulting two equivalent lenses with the corresponding separation can be once again merged into one effective lens, as given in the formula of the blog post.

**(F1)---D12--->(F2)**  ---D23---> **(F3)---D34--->(F4)**

**(Feff12)---DB12--->** ---D23---> **---DF34--->(Feff34)**

(Feff1234)

In [ ]:
_, Feff12, DB12 = equivSingleLens_2lenses(F1,D12,F2)
DF34, Feff34, _ = equivSingleLens_2lenses(F3,D34,F4)
_, Feff1234, _ = equivSingleLens_2lenses(Feff12,DB12+D23+DF34,Feff34)
expand(simplify(1/Feff1234))

  D₁₂⋅D₂₃⋅D₃₄   D₁₂⋅D₂₃    D₁₂⋅D₂₃    D₁₂⋅D₃₄    D₁₂⋅D₃₄     D₁₂     D₁₂     D
- ─────────── + ──────── + ──────── + ──────── + ──────── - ───── - ───── - ──
  F₁⋅F₂⋅F₃⋅F₄   F₁⋅F₂⋅F₄   F₁⋅F₂⋅F₃   F₁⋅F₃⋅F₄   F₁⋅F₂⋅F₄   F₁⋅F₄   F₁⋅F₃   F₁

₁₂    D₂₃⋅D₃₄    D₂₃⋅D₃₄     D₂₃     D₂₃     D₂₃     D₂₃     D₃₄     D₃₄     D
─── + ──────── + ──────── - ───── - ───── - ───── - ───── - ───── - ───── - ──
⋅F₂   F₂⋅F₃⋅F₄   F₁⋅F₃⋅F₄   F₂⋅F₄   F₂⋅F₃   F₁⋅F₄   F₁⋅F₃   F₃⋅F₄   F₂⋅F₄   F₁

₃₄    1    1    1    1 
─── + ── + ── + ── + ──
⋅F₄   F₄   F₃   F₂   F₁

Make sure the two expressions are the same.

In [ ]:
simplify(f4L_inv - 1/Feff1234)

0

# Numerical examples

You have to use consistent units for all parameters

db can be larger than the effective focal length feff. In this case, an incident collimated beam would have a focal point that can't be accessed physically because the back working distance (BWD=feff-db) would be negative, i.e., you will generate a virtual image.

In [ ]:
InaccessibleFocalP=equivSingleLens([100,50,75,150],[10,15,20])
print(InaccessibleFocalP)
InaccessibleFocalP['feff']-InaccessibleFocalP['db']

{'df': 21.43241787829833, 'feff': 30.290791599353792, 'db': 36.59127625201939}


Any parameter can be negative. In the equivalent system, a free-space propagation distance can be a negative value. This is effectively the same as a leap backwards from the beam propagation direction. Nevertheless, note that, for a collimated input beam, the focal point could be physically accessible, i.e., form a real image. This could be surprising if the effective focal length happens to be negative because we are used to the intuition that actual diverging lenses generate a virtual image.

In [ ]:
NegProp=equivSingleLens([250,10],[270])
print(NegProp)
NegProp['feff']-NegProp['db']

{'df': -6750.0, 'feff': -250.0, 'db': -270.0}


#Concatenated telescopes

Three lenses with telescope condition between them. "eps" is a dummy variable to avoid infinities in the symbolic calculation. It should be set to 0 at the end.

This special case can be easily worked out without the use of matrix mechanics, by simply looking at the behavior of a collimated beam at the input, and at the output.

(F1)---F1+F2--->(F2)---F2+F3--->(F3)

In [ ]:
eps = symbols('eps')

In [ ]:
expand(simplify((equivSingleLens([F1,F2,F3],[F1+F2+eps,F2+F3+eps])['feff']))).subs(eps,0)

In [ ]:
expand(simplify((equivSingleLens([F1,F2,F3],[F1+F2+eps,F2+F3+eps])['df']))).subs(eps,0)

      F₁⋅F₃
-F₁ - ─────
        F₂ 

In [ ]:
expand(simplify((equivSingleLens([F1,F2,F3],[F1+F2+eps,F2+F3+eps])['db']))).subs(eps,0)

How robust is a telescope image if the lenses F1 and F2 are not perfectly distanced by F1 + F2?

(F1)---F1+F2+eps--->(F2)---D23--->(F3)

Note below how the effective focal length and the front distance are stable to first order if D23 = F2 + F3. Also, the back distance becomes partially more stable under this condition

In [ ]:
diff(expand(simplify((equivSingleLens([F1,F2,F3],[F1+F2+eps,D23])['feff']))),eps).subs(eps,0)

F₁⋅F₃⋅(-D₂₃ + F₂ + F₃)
──────────────────────
           3          
         F₂           

In [ ]:
simplify(diff(expand(simplify((equivSingleLens([F1,F2,F3],[F1+F2+eps,D23])['df']))),eps).subs(eps,0))

F₁⋅(F₁⋅(-D₂₃ + F₃) + F₂⋅(F₁ + F₃))⋅(-D₂₃ + F₂ + F₃)
───────────────────────────────────────────────────
                          4                        
                        F₂                         

In [ ]:
simplify(diff(expand(simplify((equivSingleLens([F1,F2,F3],[F1+F2+eps,D23])['db']))),eps)).subs(eps,0)

   ⎛  2              ⎛          2⎞                 ⎞
F₃⋅⎝F₂ ⋅(D₂₃ - F₂) + ⎝F₁⋅F₂ + F₂ ⎠⋅(-D₂₃ + F₂ + F₃)⎠
────────────────────────────────────────────────────
                          4                         
                        F₂                          